In [1]:
import pandas as pd

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.applications import ConvNeXtTiny
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

import math
import os
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
mainPath = "/content/drive/MyDrive/Bakalaurinis/Dataset/"
# mainPath = "./dataset/"
trainPath = mainPath + "train_images/"
train_labels = pd.read_csv(mainPath + "train.csv")

In [8]:
BATCH_SIZE = 16
STEPS_PER_EPOCH = math.floor(len(train_labels)*0.8 / BATCH_SIZE)
VALIDATION_STEPS = math.floor(len(train_labels)*0.2 / BATCH_SIZE)
EPOCHS = 20
TARGET_SIZE = 512

In [9]:
train_labels.label = train_labels.label.astype('str')

train_generator = ImageDataGenerator(validation_split = 0.2,
                                     preprocessing_function = None,
                                     zoom_range = 0.2,
                                     cval = 0.2,
                                     horizontal_flip = True,
                                     vertical_flip = True,
                                     fill_mode = 'nearest',
                                     shear_range = 0.2,
                                     height_shift_range = 0.2,
                                     width_shift_range = 0.2) \
    .flow_from_dataframe(train_labels,
                         directory = trainPath,
                         subset = "training",
                         x_col = "image_id",
                         y_col = "label",
                         target_size = (TARGET_SIZE, TARGET_SIZE),
                         batch_size = BATCH_SIZE,
                         class_mode = "sparse")

validation_generator = ImageDataGenerator(validation_split = 0.2) \
    .flow_from_dataframe(train_labels,
                         directory = trainPath,
                         subset = "validation",
                         x_col = "image_id",
                         y_col = "label",
                         target_size = (TARGET_SIZE, TARGET_SIZE),
                         batch_size = BATCH_SIZE,
                         class_mode = "sparse")

Found 17118 validated image filenames belonging to 5 classes.
Found 4279 validated image filenames belonging to 5 classes.


In [10]:
base_model = ConvNeXtTiny(weights='imagenet', include_top=False, input_shape=(TARGET_SIZE, TARGET_SIZE, 3))
layers = base_model.output
layers = GlobalAveragePooling2D()(layers)
predictions = Dense(5, activation='softmax')(layers)
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

111650432/111650432 [==============================] - 1s 0us/step


In [11]:
print("Learning rate before first fit:", model.optimizer.learning_rate.numpy())

Learning rate before first fit: 0.001


In [ ]:
print(tf.config.list_physical_devices('GPU'))

In [ ]:
model_save = ModelCheckpoint(os.path.join(mainPath, 'ConvNeXtTiny.h5'), 
                             save_best_only = True, 
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.3, 
                              patience = 1, min_delta = 0.001, 
                              mode = 'min', verbose = 1)
early_stop = EarlyStopping(monitor='val_loss', patience=2, mode='min', verbose=1)


history = model.fit(
    train_generator,
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs = EPOCHS,
    validation_data = validation_generator,
    validation_steps = VALIDATION_STEPS,
    callbacks = [model_save, reduce_lr, early_stop]
)

In [ ]:
model.save(os.path.join(mainPath, 'ConvNeXtTinyFinal'))

with open(os.path.join(mainPath, 'ConvNeXtTinyHistory.pkl'), 'wb') as f:
    pickle.dump(history.history, f)